In [1]:
import requests
import pandas as pd
import ta
import time
from datetime import datetime, timedelta

# === CONFIGURATION ===
api_key = 'f5754d3325dc4149ba98663c7dd9821e'
symbol = 'XAU/USD'
interval = '1h'
start_date = '2024-01-01'
end_date = '2024-06-30'

# === Helper : télécharger une page de données ===
def fetch_data_page(start_date, end_date):
    url = 'https://api.twelvedata.com/time_series'
    params = {
        'symbol': symbol,
        'interval': interval,
        'start_date': start_date,
        'end_date': end_date,
        'apikey': api_key,
        'format': 'JSON',
        'order': 'ASC',
        'timezone': 'UTC',
        'outputsize': 5000
    }
    response = requests.get(url, params=params)
    data = response.json()
    if 'values' in data:
        return pd.DataFrame(data['values'])
    else:
        print("Erreur:", data)
        return None

# === Boucle sur l'historique ===
all_data = pd.DataFrame()
current_start = datetime.strptime(start_date, "%Y-%m-%d")
final_end = datetime.strptime(end_date, "%Y-%m-%d")

print("📥 Téléchargement des données en 1h par batch...")

while current_start < final_end:
    current_end = current_start + timedelta(days=10)
    if current_end > final_end:
        current_end = final_end

    print(f" - Téléchargement de {current_start.date()} à {current_end.date()}...")
    df_page = fetch_data_page(current_start.strftime("%Y-%m-%d"), current_end.strftime("%Y-%m-%d"))
    if df_page is not None and not df_page.empty:
        all_data = pd.concat([all_data, df_page])

    current_start = current_end + timedelta(days=1)
    time.sleep(1)

# === Nettoyage ===
if all_data.empty:
    print("❌ ERREUR : aucune donnée téléchargée.")
    exit()

print("✅ Données téléchargées. Nettoyage...")

all_data['datetime'] = pd.to_datetime(all_data['datetime'])
all_data = all_data.sort_values('datetime').reset_index(drop=True)

# Colonnes numériques
for col in ['open', 'high', 'low', 'close']:
    all_data[col] = all_data[col].astype(float)

# === Calcul des indicateurs ===
print("✅ Calcul des indicateurs techniques...")

close_series = all_data['close']

all_data['rsi'] = ta.momentum.RSIIndicator(close_series, window=10).rsi()
all_data['ema_9'] = close_series.ewm(span=9, adjust=False).mean()
all_data['ema_21'] = close_series.ewm(span=21, adjust=False).mean()
macd_calc = ta.trend.MACD(close_series)
all_data['macd_line'] = macd_calc.macd()

# === Créer les targets ===
print("✅ Création des targets (classification et régressions)...")

# Classification : binaire (up ou down)
all_data['target'] = (all_data['close'].shift(-1) > all_data['close']).astype(int)

# Régressions : amplitude attendue
all_data['target_up_move'] = all_data['close'].shift(-1) - all_data['close']
all_data['target_down_move'] = all_data['close'] - all_data['low'].shift(-1)

# Nettoyage final
all_data = all_data.dropna()

# ✅ Affichage d'exemple
print(all_data[['datetime', 'close', 'target', 'target_up_move', 'target_down_move']].head())

# === Sauvegarde CSV ===
all_data.to_csv('gold_features_hourly_with_tp_sl.csv', index=False)
print("✅ Données sauvegardées dans gold_features_hourly_with_tp_sl.csv")


📥 Téléchargement des données en 1h par batch...
 - Téléchargement de 2024-01-01 à 2024-01-11...
 - Téléchargement de 2024-01-12 à 2024-01-22...
 - Téléchargement de 2024-01-23 à 2024-02-02...
 - Téléchargement de 2024-02-03 à 2024-02-13...
 - Téléchargement de 2024-02-14 à 2024-02-24...
 - Téléchargement de 2024-02-25 à 2024-03-06...
 - Téléchargement de 2024-03-07 à 2024-03-17...
 - Téléchargement de 2024-03-18 à 2024-03-28...
 - Téléchargement de 2024-03-29 à 2024-04-08...
Erreur: {'code': 429, 'message': 'You have run out of API credits for the current minute. 9 API credits were used, with the current limit being 8. Wait for the next minute or consider switching to a higher tier plan at https://twelvedata.com/pricing', 'status': 'error'}
 - Téléchargement de 2024-04-09 à 2024-04-19...
Erreur: {'code': 429, 'message': 'You have run out of API credits for the current minute. 10 API credits were used, with the current limit being 8. Wait for the next minute or consider switching to a h

In [2]:
df = pd.read_csv('gold_features_hourly_with_tp_sl.csv')
print(df.head())


              datetime         open        high         low       close  \
0  2024-01-03 01:00:00  2060.139890  2062.61011  2059.63989  2062.46997   
1  2024-01-03 02:00:00  2062.639890  2065.21997  2061.65991  2064.76001   
2  2024-01-03 03:00:00  2064.739990  2065.65991  2063.42993  2065.15991   
3  2024-01-03 04:00:00  2065.070068  2065.63989  2064.51001  2064.92993   
4  2024-01-03 05:00:00  2064.909910  2066.01001  2063.57007  2064.13989   

         rsi        ema_9       ema_21  macd_line  target  target_up_move  \
0  46.534320  2061.784961  2064.079983  -1.938432       1         2.29004   
1  52.071399  2062.379971  2064.141804  -1.619057       1         0.39990   
2  53.015517  2062.935959  2064.234359  -1.318482       0        -0.22998   
3  52.356498  2063.334753  2064.297593  -1.086310       0        -0.79004   
4  49.984852  2063.495780  2064.283256  -0.955052       1         0.22022   

   target_down_move  
0           0.81006  
1           1.33008  
2           0.64990 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   datetime          1275 non-null   object 
 1   open              1275 non-null   float64
 2   high              1275 non-null   float64
 3   low               1275 non-null   float64
 4   close             1275 non-null   float64
 5   rsi               1275 non-null   float64
 6   ema_9             1275 non-null   float64
 7   ema_21            1275 non-null   float64
 8   macd_line         1275 non-null   float64
 9   target            1275 non-null   int64  
 10  target_up_move    1275 non-null   float64
 11  target_down_move  1275 non-null   float64
dtypes: float64(10), int64(1), object(1)
memory usage: 119.7+ KB


In [4]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error
import pickle

# ========== 1️⃣ Charger les données ==========

df = pd.read_csv('gold_features_hourly_with_tp_sl.csv')
print("✅ Données chargées")
print(df.head())

# ========== 2️⃣ Sélection des features ==========

features = [
    'rsi', 'ema_9', 'ema_21', 'macd_line'
]

X = df[features]
y_class = df['target']
y_tp = df['target_up_move']
y_sl = df['target_down_move']

# ========== 3️⃣ Split des données ==========

X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X, y_class, test_size=0.2, random_state=42, stratify=y_class
)

X_train_reg, X_test_reg, y_train_tp, y_test_tp, y_train_sl, y_test_sl = train_test_split(
    X, y_tp, y_sl, test_size=0.2, random_state=42
)

print("✅ Split des données effectué")

# ========== 4️⃣ Entraîner les modèles ==========

## 🔹 Classifier (hausse ou baisse)
clf_model = RandomForestClassifier(n_estimators=100, random_state=42)
clf_model.fit(X_train_cls, y_train_cls)

## 🔹 Regressor pour TP
tp_model = RandomForestRegressor(n_estimators=100, random_state=42)
tp_model.fit(X_train_reg, y_train_tp)

## 🔹 Regressor pour SL
sl_model = RandomForestRegressor(n_estimators=100, random_state=42)
sl_model.fit(X_train_reg, y_train_sl)

print("✅ Entraînement terminé")

# ========== 5️⃣ Evaluation rapide ==========

## 📌 Classifier
y_pred_cls = clf_model.predict(X_test_cls)
print("\n✅ Classifier Accuracy:", accuracy_score(y_test_cls, y_pred_cls))
print("\n✅ Classification Report:\n", classification_report(y_test_cls, y_pred_cls))

## 📌 TP Regressor
y_pred_tp = tp_model.predict(X_test_reg)
print("\n✅ MAE Take-Profit:", mean_absolute_error(y_test_tp, y_pred_tp))

## 📌 SL Regressor
y_pred_sl = sl_model.predict(X_test_reg)
print("\n✅ MAE Stop-Loss:", mean_absolute_error(y_test_sl, y_pred_sl))

# ========== 6️⃣ Sauvegarde des modèles ==========

with open('ict_classifier.pkl', 'wb') as f:
    pickle.dump(clf_model, f)

with open('ict_tp_model.pkl', 'wb') as f:
    pickle.dump(tp_model, f)

with open('ict_sl_model.pkl', 'wb') as f:
    pickle.dump(sl_model, f)

print("\n✅ Tous les modèles sauvegardés avec succès")


✅ Données chargées
              datetime         open        high         low       close  \
0  2024-01-03 01:00:00  2060.139890  2062.61011  2059.63989  2062.46997   
1  2024-01-03 02:00:00  2062.639890  2065.21997  2061.65991  2064.76001   
2  2024-01-03 03:00:00  2064.739990  2065.65991  2063.42993  2065.15991   
3  2024-01-03 04:00:00  2065.070068  2065.63989  2064.51001  2064.92993   
4  2024-01-03 05:00:00  2064.909910  2066.01001  2063.57007  2064.13989   

         rsi        ema_9       ema_21  macd_line  target  target_up_move  \
0  46.534320  2061.784961  2064.079983  -1.938432       1         2.29004   
1  52.071399  2062.379971  2064.141804  -1.619057       1         0.39990   
2  53.015517  2062.935959  2064.234359  -1.318482       0        -0.22998   
3  52.356498  2063.334753  2064.297593  -1.086310       0        -0.79004   
4  49.984852  2063.495780  2064.283256  -0.955052       1         0.22022   

   target_down_move  
0           0.81006  
1           1.33008  
2